# Stochastic Simulation

*Winter Semester 2023/24*

03.11.2023

Prof. Sebastian Krumscheid<br>
Asstistant: Stjepan Salatovic

<h3 align="center">
Exercise sheet 01
</h3>

---

<h1 align="center">
Random Number Generation
</h1>

In [1]:
import matplotlib.pylab as plt
import numpy as np

from scipy.stats import uniform
from ipywidgets import interact

## Exercise 1

Consider **SciPy's** default **uniform** random number generator
(RNG) `uniform` within the Statistics module `scipy.stats` and use it to generate a sequence of numbers $U_1, U_2, \dots, U_n$. Have a look at its [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) and the [full list](https://docs.scipy.org/doc/scipy/reference/stats.html) of available distributions and other statistical functionalities in `scipy.stats`. Consider different values of $n$, for example $n=25,100,10^3,10^5$, and address the following points:

1. Plot the cumulative distribution function (CDF) of the theorized uniform distribution $\mathcal{U}(0,1)$ together with the empirical CDF of the data. Furthermore, produce a Q-Q plot of the data. Use both plots to assess the quality of the sequence with respect to the theorized $\mathcal{U}(0,1)$ distribution. Describe your observations.

**Hint:** The function [`plt.step`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.step.html) might come in handy.

In [2]:
def cdf(seq: np.array, x: np.array) -> np.array:
    """Computes the empirical CDF of `seq` and evaluates in `x`."""
    n = len(seq)
    indices = np.searchsorted(np.sort(seq), x, side='right')
    y = np.concatenate(([0], np.arange(1, n + 1) / n))
    return y[indices]

In [3]:
def cdf_qq_plot(seq: np.array):
    """
    Plots the empirical CDF and a QQ plot of the sample `seq`
    and compares with theoretical uniform one.
    """
    n = len(seq)
    
    x = np.linspace(0, 1, 1000)

    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

    axs[0].plot([0, 1], [0, 1], label="CDF")
    axs[0].step(x, cdf(seq, x), label="Empirical CDF")
    axs[0].set_title("CDF")
    axs[0].legend()

    axs[1].step(np.arange(1, n + 1) / n, np.sort(seq), "o")
    axs[1].plot([0, 1], [0, 1], "k--")
    axs[1].set_title("QQ-Plot");

The following interactive plot illustrates the CDF comparisons and Q-Q plots for a rand sequence (seed is fixed for reproducibility; c.f. code). We see that increasing $n$ results in a better linear behavior in the Q-Q plots as well as a better fit of the empirical CDF and the theorized CDF.

In [4]:
def uniform_rng_scipy_plots(n: int):
    """Interaction helper."""
    np.random.seed(999)

    seq = uniform.rvs(size=n)
    cdf_qq_plot(seq)

interact(uniform_rng_scipy_plots, n=[25, 100, 10**3, 10 **5]);

interactive(children=(Dropdown(description='n', options=(25, 100, 1000, 100000), value=25), Output()), _dom_cl…

2. Implement the Kolmogorov-Smirnov test to ascertain whether the empirical CDF of the sample $U_1, U_2, \dots, U_n$ matches the theoretical CDF of the $\mathcal{U}(0,1)$ distribution at level $\alpha=0.1$. That is, we reject the null hypothesis $H_0$ at level $\alpha>0$ that the sample $U_1,\dots,U_n\overset{\text{iid}}{\sim} \mathcal{U}(0,1)$ if $\sqrt{n}D_n>K_{\alpha,n}$, where $D_n={\sup}_{x\in \mathbb{R}}| \hat F(x)- F(x)|,$ and $K_{\alpha,n}$ is such that $\mathbb{P}(\sqrt{n}D_n>K_{\alpha,n})<\alpha$.

    **Note:** It is known that the appropriately scaled test statistic $D_n$ converges in distribution to a Kolmogorov random variable $K_{\alpha, \infty}$ independently of $F$, where $\mathbb{P}(K_{\alpha, \infty} \le x) = 1+2\sum_{j=1}^\infty{(-1)}^je^{-2j^2x^2}$, $x>0$. This asymptotic result can then be used to compute the required $1-\alpha$ quantiles of $K_{\alpha, n}$ approximately by using $K_{\alpha,n}\simeq K_{\alpha,\infty}$ for $n \gg 1$. It is however also possible to characterize the distribution of $D_n$ directly, which is useful for small values of $n$. The following Table presents some of these pre-asymptotic $1 − \alpha$ quantiles $K_{\alpha, n}$.
    
| | $\alpha$ | $0.20$ | $0.10$ | $0.05$ | $0.01$ |
|---|---|---|---|---|---|
|$n$||||||
| 1 || 0\.90 | 0\.95 | 0\.98 | 0\.99 |
| 2 || 0\.96 | 1\.10 | 1\.19 | 1\.32 |
| 3 || 0\.97 | 1\.11 | 1\.23 | 1\.44 |
| 4 || 0\.98 | 1\.12 | 1\.24 | 1\.46 |
| 5 || 1\.01 | 1\.14 | 1\.25 | 1\.50 |
| 6 || 1\.00 | 1\.15 | 1\.27 | 1\.52 |
| 7 || 1\.01 | 1\.16 | 1\.30 | 1\.53 |
| 8 || 1\.02 | 1\.16 | 1\.30 | 1\.53 |
| 9 || 1\.02 | 1\.17 | 1\.29 | 1\.53 |
| 10 || 1\.01 | 1\.17 | 1\.30 | 1\.55 |
| 11 || 1\.03 | 1\.16 | 1\.29 | 1\.56 |
| 12 || 1\.04 | 1\.18 | 1\.32 | 1\.56 |
| 15 || 1\.05 | 1\.16 | 1\.32 | 1\.55 |
| 20 || 1\.03 | 1\.16 | 1\.30 | 1\.57 |
| 30 || 1\.04 | 1\.20 | 1\.31 | 1\.59 |
| 35 || 1\.06 | 1\.24 | 1\.36 | 1\.60 |
| 40 || 1\.08 | 1\.20 | 1\.33 | 1\.58 |
| 45 || 1\.07 | 1\.21 | 1\.34 | 1\.61 |
| $n>45$ || 1.07 | 1.22 | 1.36 | 1.63 |
    
**Tip:** Instead of the table, you can also use [`scipy.stats.kstwo`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstwo.html) to get pre-asymptotic $1-\alpha$ quantiles $K_{\alpha,n}$. For instance, to get the $1 - \alpha$ quantile of the two-sided Kolmogorov-Smirnov test statistic with $n$ samples, use the command `np.sqrt(n) * kstwo(n).ppf(1 - alpha)`.

The function `ppf` is the percent point function (inverse of CDF) of a random variable in **SciPy**. For a sample of size $n > 45$, use the asymptotic distribution of the test statistic for better results: `scipy.stats.kstwobign` (no need for specifying `n` here).

In [5]:
from scipy.stats import kstwo, kstwobign

In [6]:
def kolmogorov_smirnov(seq: np.array, alpha: float=.1) -> bool:
    """
    Kolmogorov Smirnov test for data `seq` and significance `alpha`.
    Returns `True` if H0 cannot be rejected and `False` if rejected at level `alpha`.
    """
    n = len(seq)

    x = np.linspace(0, 1, 10000)
    D_n = np.max(np.abs(x - cdf(seq, x)))
    
    if n <= 45:
        if np.sqrt(n) * D_n > np.sqrt(n) * kstwo(n).ppf(1 - alpha):
            return False
    else:
        if np.sqrt(n) * D_n > kstwobign().ppf(1 - alpha):
            return False

    return True

The test outcomes $(\alpha = 0.1)$ for the same sequences are shown in the following output, suggesting that `scipy.stats`’s default RNG produces a sequence with the desired uniform distribution indeed.

In [7]:
from typing import Callable, Literal

In [8]:
def acc_test(
    test: Callable,
    rng: Literal={"Scipy", "LCG"},
    alpha: float=.1,
    reps: int=100,
    ns: list=[25, 100, 10**3, 10**5]
):
    """
    Evaluates the quality of `test` by repeating a specific number (`reps`) of tests.
    """
    test_2_name = {}

    try: test_2_name[kolmogorov_smirnov] = "Kolmogorov-Smirnov"
    except: pass
    try: test_2_name[chi_squared] = "Chi-squared"
    except: pass
    try: test_2_name[serial_test] = "Serial"
    except: pass
    try: test_2_name[gap_test] = "Gap"
    except: pass
        
    print(f"Test: {test_2_name[test]}")
    print(f"RNG: {rng}\n***")
    print(f"Results based on {reps} tests with alpha = {alpha}:")
    print("-" * 48)

    for n in ns:
        results = []
        for _ in range(reps):
            if rng == "Scipy":
                unif_seq = uniform.rvs(size=n)
            elif rng == "LCG":
                unif_seq = LCG(n=n)
            else:
                print(f"RNG '{rng}' not implemented. Choose 'Scipy' or 'LCG'.")
            test_outcome = test(unif_seq, alpha=alpha)
            results.append(test_outcome)
        acc = np.sum(results) / reps
        print(f"n = {n:<8}: {acc * 100}% of tests cannot be rejected")

In [9]:
acc_test(kolmogorov_smirnov, rng="Scipy")

Test: Kolmogorov-Smirnov
RNG: Scipy
***
Results based on 100 tests with alpha = 0.1:
------------------------------------------------
n = 25      : 88.0% of tests cannot be rejected
n = 100     : 94.0% of tests cannot be rejected
n = 1000    : 87.0% of tests cannot be rejected
n = 100000  : 84.0% of tests cannot be rejected


3. Implement the $\chi^2$ goodness of fit test to ascertain whether the sequence $U_1, U_2, \dots, U_n$ is uniformly distributed. A description of such method can be found on section 8.7.4 of _Handbook of Monte Carlo Methods_ (see also Section 1.2.1 of the lecture notes). 
  
    **Hint:** Again, you can use the `ppf` function of the `scipy.stats.chi2` class to compute quantiles of a $\chi^2$ distribution.

In [10]:
from scipy.stats import chi2

In [11]:
def chi_squared(seq: np.array, K: int=10, alpha: float=.1) -> bool:
    """
    Chi-Squared test for data `seq` and significance `alpha`.
    Returns `True` if H0 cannot be rejected and `False` if rejected at level `alpha`.
    """
    n = len(seq)
    N, _ = np.histogram(seq, bins=np.arange(K + 1) / K)
    Q = np.sum((N - n / K) ** 2 / (n / K))
    if Q > chi2.ppf(1 - alpha, K - 1):
        return False
    return True

In [12]:
acc_test(chi_squared, rng="Scipy")

Test: Chi-squared
RNG: Scipy
***
Results based on 100 tests with alpha = 0.1:
------------------------------------------------
n = 25      : 92.0% of tests cannot be rejected
n = 100     : 91.0% of tests cannot be rejected
n = 1000    : 90.0% of tests cannot be rejected
n = 100000  : 91.0% of tests cannot be rejected


4.  Repeat the tests in points 2. and 3. for different values of $\alpha$. What do you observe? Explain your findings.

The smaller the values of $\alpha$, the higher the confidence with which we require the test to hold. To account for an increased level of confidence, the value of a test statistic has to be even bigger before a test rejects the null hypothesis. This can be observed in the output of the following cell.

In [13]:
alphas = [.1, .05, .01]

for alpha in alphas:
    acc_test(kolmogorov_smirnov, rng="Scipy", alpha=alpha)
    print("\n")

Test: Kolmogorov-Smirnov
RNG: Scipy
***
Results based on 100 tests with alpha = 0.1:
------------------------------------------------
n = 25      : 90.0% of tests cannot be rejected
n = 100     : 95.0% of tests cannot be rejected
n = 1000    : 85.0% of tests cannot be rejected
n = 100000  : 88.0% of tests cannot be rejected


Test: Kolmogorov-Smirnov
RNG: Scipy
***
Results based on 100 tests with alpha = 0.05:
------------------------------------------------
n = 25      : 96.0% of tests cannot be rejected
n = 100     : 98.0% of tests cannot be rejected
n = 1000    : 92.0% of tests cannot be rejected
n = 100000  : 99.0% of tests cannot be rejected


Test: Kolmogorov-Smirnov
RNG: Scipy
***
Results based on 100 tests with alpha = 0.01:
------------------------------------------------
n = 25      : 100.0% of tests cannot be rejected
n = 100     : 98.0% of tests cannot be rejected
n = 1000    : 99.0% of tests cannot be rejected
n = 100000  : 100.0% of tests cannot be rejected




## Exercise 2

Implement the linear congruential generator (LCG) 
\begin{equation*}
  X_k = (aX_{k-1} + b) \bmod m\;,\quad U_k := \frac{X_k}{m}\;,
\end{equation*}
with $a=3$, $b=0$, and $m=31$.

In [14]:
def LCG(x0: float=1, n: int=100, a: int=3, b: int=0, m: int=31) -> np.array:
    """
    Generates `n` numbers using the linear congruential generator.
    """
    x = np.zeros(n)
    x[0] = x0
    for i in range(1, n):
        x[i] = (a * x[i-1] + b) % m

    U = x / m
    return U

1. Use your LCG procedure to generate a sequence $U_1, U_2, \dots, U_n$ and repeat Exercise 1. Discuss your results.

In [15]:
def lcg_plots(n: int):
    """Interaction helper."""
    seq = LCG(n=n)
    cdf_qq_plot(seq)

interact(lcg_plots, n=[25, 100, 10**3, 10 **5]);

interactive(children=(Dropdown(description='n', options=(25, 100, 1000, 100000), value=25), Output()), _dom_cl…

We see that the KS-test rejects the null hypothesis of a uniform
distribution for large values of $n$. This is a consequence of the
non-vanishing residual of $\lvert \hat{F}-F\rvert$ due to the
periodicity of the LCG sequence ($m=31$ is small).

Notice that the $\chi^2$-test with number of bins $K=10$ does not
reject the null hypothesis even for large values of $n$. This is due
to the fact that $K$ is small compared to the length of the sequence
$n$. Increasing the value of $K$ (e.g., $K=20$) for large values of
$n$ changes this (see also the discussion on the value of $r$
below).

In [16]:
alphas = [.1, .05, .01]

for alpha in alphas:
    acc_test(kolmogorov_smirnov, rng="LCG", alpha=alpha)
    print("\n")
    acc_test(chi_squared, rng="LCG", alpha=alpha)
    print("\n")

Test: Kolmogorov-Smirnov
RNG: LCG
***
Results based on 100 tests with alpha = 0.1:
------------------------------------------------
n = 25      : 100.0% of tests cannot be rejected
n = 100     : 100.0% of tests cannot be rejected
n = 1000    : 100.0% of tests cannot be rejected
n = 100000  : 0.0% of tests cannot be rejected


Test: Chi-squared
RNG: LCG
***
Results based on 100 tests with alpha = 0.1:
------------------------------------------------
n = 25      : 100.0% of tests cannot be rejected
n = 100     : 100.0% of tests cannot be rejected
n = 1000    : 100.0% of tests cannot be rejected
n = 100000  : 100.0% of tests cannot be rejected


Test: Kolmogorov-Smirnov
RNG: LCG
***
Results based on 100 tests with alpha = 0.05:
------------------------------------------------
n = 25      : 100.0% of tests cannot be rejected
n = 100     : 100.0% of tests cannot be rejected
n = 1000    : 100.0% of tests cannot be rejected
n = 100000  : 0.0% of tests cannot be rejected


Test: Chi-squared
RN

2. Explain why one would expect that the Serial test (with $d=2$, say) is an appropriate test to scrutinize the LCG. Support your explanation by applying the Serial test at level $\alpha=0.1$ to sequences (for various values of $n$) from both the LCG and from the default `scipy.stats` RNG `uniform`.

In [17]:
def serial_test(seq: np.array, d: int=2, alpha: float=.1) -> bool:
    """
    Serial test for data `seq` and significance `alpha`.
    Returns `True` if H0 cannot be rejected and `False` if rejected at level `alpha`.
    """
    assert seq.shape[0] % d == 0, 'Random sample length should be divisible by `d`.'
    nn = int(seq.shape[0] / d)
    m = 10
    K = m ** d
    N = np.histogramdd(seq.reshape(d, -1).T, bins=m)[0].reshape(-1)
    Q = np.sum((N - nn / K) ** 2 / (nn / K))
    if Q > chi2.ppf(1 - alpha, K - 1):
        return False
    return True

Note that we used `numpy.histogramdd` in the definition of `serial_test` to make life easier. If you would want to construct the variable `N` yourself, it could look like this:
```python
Y = seq.reshape(2, int(seq.shape[0]/2)).T
N = np.zeros(K)
p = np.ones(K) / float(K) # True probabilities (uniform partition)
for k in range(K):
    xl = (k % m) / float(m)
    xu = xl + 1./m
    yl = np.max([0., np.floor(k/float(m))]) / m
    yu = yl + 1./m
    N[k] = np.sum( ((xl< Y[:,0]) & (Y[:,0] <= xu)) * ( (yl<Y[:,1]) & (Y[:,1]<= yu)))
```

In [18]:
acc_test(serial_test, rng="Scipy", ns=[26, 10**2, 10**3, 10**4])
print("\n")
acc_test(serial_test, rng="LCG", ns=[26, 10**2, 10**3, 10**4])

Test: Serial
RNG: Scipy
***
Results based on 100 tests with alpha = 0.1:
------------------------------------------------
n = 26      : 82.0% of tests cannot be rejected
n = 100     : 91.0% of tests cannot be rejected
n = 1000    : 92.0% of tests cannot be rejected
n = 10000   : 83.0% of tests cannot be rejected


Test: Serial
RNG: LCG
***
Results based on 100 tests with alpha = 0.1:
------------------------------------------------
n = 26      : 100.0% of tests cannot be rejected
n = 100     : 0.0% of tests cannot be rejected
n = 1000    : 0.0% of tests cannot be rejected
n = 10000   : 0.0% of tests cannot be rejected


The reason why we expect the Serial test to perform very well in rejecting the LCG sequence is the LCG's periodicity. Specifically, the following figure shows plots of the tuple $(U_i,U_{i+1})$ for both a `scipy.uniform` sequence (blue) and a `LCG` (orange) sequence.

While we cannot recognize any systematic pattern in the plot for
the `scipy` generated numbers, the plot for the LCG shows a systematic
pattern. As a consequence, the full state space is not explored by
the sequence. The serial test is, however, exactly designed for
ascertain a uniform coverage of the state space, which suggests
that the test will reject the LCG sequences.
The implementation of the Serial test is a simple extension of the
$\chi^2$-test used in the previous Exercise.

Here the serial test is implemented with $m=10$ subdivisions along
each dimension. The numerical tests confirm the motivation that
this test is effective to identify the `LCG` sequence as not coming
from a uniform random variable. In fact, the test rejects the null
hypothesis at level $\alpha=0.1$ already for $n\ge 100$; see above cell.
A rule-of-thumb in practice is to use $n$ and $m$ such that $n\ge 5 m^d$.
This is satisfied here for $n\ge 1000$ ($m=10$ and $d=2$).

In [19]:
def plot_scipy_vs_lcg_space(n: int):
    """Interaction helper."""
    x = uniform.rvs(size=n)
    plt.plot(x[:-1], x[1:], lw=.5, label="scipy")

    x = LCG(n=n)
    plt.plot(x[:-1], x[1:], lw=1, label="LCG")
    
    plt.xlabel(r"$U_i$")
    plt.xlabel(r"$U_{i + 1}$")
    plt.axis("square")
    plt.legend(fontsize=15)

interact(plot_scipy_vs_lcg_space, n=(10, 1000));

interactive(children=(IntSlider(value=505, description='n', max=1000, min=10), Output()), _dom_classes=('widge…

3. Implement the Gap test. Apply the test to both a sequence obtained from the default `scipy.stats` RNG `uniform` and to a sequence generated by the LCG. What do you observe?

In [20]:
def gap_test(seq: np.array, alpha: float=.1, a: float=0., b: float=.5, r: int=5) -> bool:
    """
    Gap test for data `seq` and significance `alpha`.
    Returns `True` if H0 cannot be rejected and `False` if rejected at level `alpha`.
    """
    idx = np.where((a < seq) & (seq < b))[0]
    idx = np.hstack([0, np.array(idx) + 1])
    
    Z = idx[1:] - idx[:-1] - 1
    nZ = len(Z)

    Nr = np.bincount(Z, minlength=r)
    Nr = np.hstack((Nr[:r], Nr[r:].sum()))

    prob = b - a
    p = prob * (1 - prob) ** np.arange(r)
    p = np.hstack([p, (1 - prob) ** r])

    Q = np.sum((Nr - p * nZ) ** 2 / (p * nZ))

    if Q > chi2.ppf(1 - alpha, r):
        return False
    return True

In [21]:
acc_test(gap_test, rng="Scipy")
print("\n")
acc_test(gap_test, rng="LCG")

Test: Gap
RNG: Scipy
***
Results based on 100 tests with alpha = 0.1:
------------------------------------------------
n = 25      : 85.0% of tests cannot be rejected
n = 100     : 96.0% of tests cannot be rejected
n = 1000    : 88.0% of tests cannot be rejected
n = 100000  : 89.0% of tests cannot be rejected


Test: Gap
RNG: LCG
***
Results based on 100 tests with alpha = 0.1:
------------------------------------------------
n = 25      : 100.0% of tests cannot be rejected
n = 100     : 0.0% of tests cannot be rejected
n = 1000    : 0.0% of tests cannot be rejected
n = 100000  : 0.0% of tests cannot be rejected


Here the Gap-test is implemented with $r=5$ and the test outcomes are shown in the cell above. We observe that the Gap-test rejects the null hypothesis at the level $\alpha=0.1$ for $n\ge 100$
of the LCG sequence, while the null hypothesis for the `uniform.rvs` sequence cannot be rejected most of the time. Notice however, that the smallest expected number per class is not bigger than $5$ (which is the
recommended threshold in practice) for approximately $n<350$, so that the outcomes for smaller values of $n$ may not be reliable.

**Comment on built-in functions**

Many of the tasks that need to be implemented for this exercise sheet already exist as **Python** built-in functions. 
For example, the empirical CDF can be conveniently plotted using the `ECDF` function that is available in the [statsmodel](https://pypi.org/project/statsmodels/) package. The [Kolmogorv-Smirnov](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html) and [$\chi^2$ test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html) are also available. 
There is, of course, very little reason to reinvent the wheel, and we strongly encourage you to use these built-in functions in future exercise sheets, if not stated otherwise. However, before naively relying on built-in functions, it is important to understand the underlying mathematical procedure.